In [29]:
# Snorkel Introduction

from glob import glob
import os

import pandas as pd
import snorkel
import tensorflow as tf

In [8]:
TAG_LIMIT = 2000
BAD_LIMIT = 500

In [2]:
%matplotlib inline

# Make sure we're running from the spam/ directory
if os.path.basename(os.getcwd()) == "snorkel-tutorials":
    os.chdir("spam")

# Turn off TensorFlow logging messages
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# For reproducibility
os.environ["PYTHONHASHSEED"] = "0"

In [24]:
df = pd.DataFrame()
path = '../data/stackoverflow/PerTag.Bad.{}.{}.parquet'.format(
    TAG_LIMIT,
    BAD_LIMIT,
)

# This takes a minute because there are 1800+ directories of JSON Lines files to load
for filename in glob(path):
    new_df = pd.read_parquet(filename)
    df = pd.concat([df, new_df])
    
    # Manage memory explicitly - although these files are tiny, its a good habit
    del new_df

df.head()

../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1181.phpexcel.jsonl/part-00009-21dc90ef-9aec-485f-9660-a0e840b8ecd9-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1181.phpexcel.jsonl/part-00004-21dc90ef-9aec-485f-9660-a0e840b8ecd9-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1181.phpexcel.jsonl/part-00010-21dc90ef-9aec-485f-9660-a0e840b8ecd9-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1181.phpexcel.jsonl/part-00003-21dc90ef-9aec-485f-9660-a0e840b8ecd9-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1181.phpexcel.jsonl/part-00007-21dc90ef-9aec-485f-9660-a0e840b8ecd9-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1181.phpexcel.jsonl/part-00000-21dc90ef-9aec-485f-9660-a0e840b8ecd9-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1181.phpexcel.jsonl/part-00002-21dc90ef-9aec-485f-9660-a0e840b8ecd9-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1181.phpexcel.jsonl/part-00011-21dc90ef-9aec-485f-9660-a0e840b8ecd9-c0

../data/stackoverflow/PerTag.Bad.2000.500.jsonl/30.admob.jsonl/part-00007-5422c4a5-52ae-423c-995e-e7b9f7dc2454-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/30.admob.jsonl/part-00010-5422c4a5-52ae-423c-995e-e7b9f7dc2454-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/30.admob.jsonl/part-00003-5422c4a5-52ae-423c-995e-e7b9f7dc2454-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/30.admob.jsonl/part-00009-5422c4a5-52ae-423c-995e-e7b9f7dc2454-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/30.admob.jsonl/part-00004-5422c4a5-52ae-423c-995e-e7b9f7dc2454-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/198.benchmarking.jsonl/part-00008-d8cb66fe-79b6-4a2e-a7ce-81071c69e913-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/198.benchmarking.jsonl/part-00005-d8cb66fe-79b6-4a2e-a7ce-81071c69e913-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/198.benchmarking.jsonl/part-00011-d8cb66fe-79b6-4a2e-a7ce-81071c69e913-c000.json
../data/

../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1558.system.jsonl/part-00002-fe09835a-031c-4bd9-ba5f-aa550fc9f25f-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1558.system.jsonl/part-00005-fe09835a-031c-4bd9-ba5f-aa550fc9f25f-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1558.system.jsonl/part-00008-fe09835a-031c-4bd9-ba5f-aa550fc9f25f-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1558.system.jsonl/part-00001-fe09835a-031c-4bd9-ba5f-aa550fc9f25f-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1558.system.jsonl/part-00006-fe09835a-031c-4bd9-ba5f-aa550fc9f25f-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1569.telerik.jsonl/part-00000-6196dbb8-0602-4f4b-9e3d-6a6c1fcdd05b-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1569.telerik.jsonl/part-00007-6196dbb8-0602-4f4b-9e3d-6a6c1fcdd05b-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1569.telerik.jsonl/part-00003-6196dbb8-0602-4f4b-9e3d-6a6c1fcdd05b-c000.json
../da

../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1293.raphael.jsonl/part-00008-72f3ce96-ee56-45d9-ab4f-f3e367e5d387-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1293.raphael.jsonl/part-00005-72f3ce96-ee56-45d9-ab4f-f3e367e5d387-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1293.raphael.jsonl/part-00011-72f3ce96-ee56-45d9-ab4f-f3e367e5d387-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1293.raphael.jsonl/part-00002-72f3ce96-ee56-45d9-ab4f-f3e367e5d387-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1293.raphael.jsonl/part-00000-72f3ce96-ee56-45d9-ab4f-f3e367e5d387-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1293.raphael.jsonl/part-00007-72f3ce96-ee56-45d9-ab4f-f3e367e5d387-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1293.raphael.jsonl/part-00003-72f3ce96-ee56-45d9-ab4f-f3e367e5d387-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1293.raphael.jsonl/part-00010-72f3ce96-ee56-45d9-ab4f-f3e367e5d387-c000.json


../data/stackoverflow/PerTag.Bad.2000.500.jsonl/150.atomic.jsonl/part-00009-05b8a924-2deb-4d45-910b-a4eef64f35ad-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/150.atomic.jsonl/part-00004-05b8a924-2deb-4d45-910b-a4eef64f35ad-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/150.atomic.jsonl/part-00006-05b8a924-2deb-4d45-910b-a4eef64f35ad-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/150.atomic.jsonl/part-00001-05b8a924-2deb-4d45-910b-a4eef64f35ad-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/150.atomic.jsonl/part-00005-05b8a924-2deb-4d45-910b-a4eef64f35ad-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/150.atomic.jsonl/part-00008-05b8a924-2deb-4d45-910b-a4eef64f35ad-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/150.atomic.jsonl/part-00002-05b8a924-2deb-4d45-910b-a4eef64f35ad-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/150.atomic.jsonl/part-00011-05b8a924-2deb-4d45-910b-a4eef64f35ad-c000.json
../data/stackove

...

../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1268.python-multiprocessing.jsonl/part-00005-29c0c2bf-f575-47d9-adb3-e136e779fda1-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/948.log4net.jsonl/part-00005-5962344f-3879-4db4-b1dd-9177271993e1-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/948.log4net.jsonl/part-00008-5962344f-3879-4db4-b1dd-9177271993e1-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/948.log4net.jsonl/part-00002-5962344f-3879-4db4-b1dd-9177271993e1-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/948.log4net.jsonl/part-00011-5962344f-3879-4db4-b1dd-9177271993e1-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/948.log4net.jsonl/part-00006-5962344f-3879-4db4-b1dd-9177271993e1-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/948.log4net.jsonl/part-00001-5962344f-3879-4db4-b1dd-9177271993e1-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/948.log4net.jsonl/part-00010-5962344f-3879-4db4-b1dd-9177271993e1-c0

../data/stackoverflow/PerTag.Bad.2000.500.jsonl/314.collation.jsonl/part-00001-89194055-8596-4df8-a261-d2b1be7925fe-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/314.collation.jsonl/part-00006-89194055-8596-4df8-a261-d2b1be7925fe-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/314.collation.jsonl/part-00009-89194055-8596-4df8-a261-d2b1be7925fe-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/314.collation.jsonl/part-00004-89194055-8596-4df8-a261-d2b1be7925fe-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/314.collation.jsonl/part-00010-89194055-8596-4df8-a261-d2b1be7925fe-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/314.collation.jsonl/part-00003-89194055-8596-4df8-a261-d2b1be7925fe-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/314.collation.jsonl/part-00007-89194055-8596-4df8-a261-d2b1be7925fe-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/314.collation.jsonl/part-00000-89194055-8596-4df8-a261-d2b1be7925fe-c0

../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1539.substring.jsonl/part-00006-f501d14b-ff02-4941-b80b-3be488d397b1-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1539.substring.jsonl/part-00001-f501d14b-ff02-4941-b80b-3be488d397b1-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1632.typo3.jsonl/part-00005-2694e456-f622-43e7-adb4-598d10490d86-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1632.typo3.jsonl/part-00008-2694e456-f622-43e7-adb4-598d10490d86-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1632.typo3.jsonl/part-00002-2694e456-f622-43e7-adb4-598d10490d86-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1632.typo3.jsonl/part-00011-2694e456-f622-43e7-adb4-598d10490d86-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1632.typo3.jsonl/part-00006-2694e456-f622-43e7-adb4-598d10490d86-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1632.typo3.jsonl/part-00001-2694e456-f622-43e7-adb4-598d10490d86-c000.json
../data/

../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1323.render.jsonl/part-00002-df0ab604-9ff6-4051-b465-062cd86a8906-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1323.render.jsonl/part-00011-df0ab604-9ff6-4051-b465-062cd86a8906-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1323.render.jsonl/part-00006-df0ab604-9ff6-4051-b465-062cd86a8906-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1323.render.jsonl/part-00001-df0ab604-9ff6-4051-b465-062cd86a8906-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1354.roslyn.jsonl/part-00000-3690e562-6151-4d27-8e36-4913b92f2553-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1354.roslyn.jsonl/part-00007-3690e562-6151-4d27-8e36-4913b92f2553-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1354.roslyn.jsonl/part-00003-3690e562-6151-4d27-8e36-4913b92f2553-c000.json
../data/stackoverflow/PerTag.Bad.2000.500.jsonl/1354.roslyn.jsonl/part-00010-3690e562-6151-4d27-8e36-4913b92f2553-c000.json
../data/

,_Body,_Index,_Tag
0,"[How, I, get, range, cells, PHPExcel, I, using...",1181,phpexcel
1,"[Workaround, copying, style, PHPExcel, I, want...",1181,phpexcel
2,"[How, set, active, sheet, without, loading, xl...",1181,phpexcel
3,"[PHPExcel, GetCalculatedValue, return, wrong, ...",1181,phpexcel
4,"[PHPExcel, Invalid, cell, coordinate, A, inser...",1181,phpexcel


In [26]:
pd.set_option('display.max_colwidth', 300)
df.head()

,_Body,_Index,_Tag
0,"[How, I, get, range, cells, PHPExcel, I, using, PHPExcel, read, data, file, I, want, get, couple, cells, say, A6, A11, I, know, I, use, get, single, cell, I, could, probably, loop, array, get, cell, range, But, simpler, method, get, range, cells, something, like, __PAD__, __PAD__, __PAD__, __PAD...",1181,phpexcel
1,"[Workaround, copying, style, PHPExcel, I, want, copy, style, information, cells, ranges, like, Format, Painter, Excel, The, documentation, says, something, like, There, appears, bug, D1, D100, E1, E100, get, style, cell, B1, If, I, change, order, two, lines, ranges, get, style, A1, Similarly, re...",1181,phpexcel
2,"[How, set, active, sheet, without, loading, xlsx, file, I, using, generate, xl, using, php, I, loading, xl, sheet, creating, new, sheets, using, I, want, set, active, sheet, using, phpExcel, using, im, getting, error, Please, help, __PAD__, __PAD__, __PAD__, __PAD__, __PAD__, __PAD__, __PAD__, _...",1181,phpexcel
3,"[PHPExcel, GetCalculatedValue, return, wrong, value, I, xlsx, file, 6, sheet, I, load, When, I, want, get, calculated, value, cell, I, get, wrong, value, This, cell, formula, In, PHP, code, I, When, I, debbuging, I, obtain, Can, help, please, P, S, Sorry, english, __PAD__, __PAD__, __PAD__, __PA...",1181,phpexcel
4,"[PHPExcel, Invalid, cell, coordinate, A, inserting, Excel, SQL, database, In, PHP, I, looping, excel, file, inserting, MSSQL, database, I, getting, error, Uncaught, exception, PHPExcel_Exception, message, Invalid, cell, coordinate, A, I, get, error, I, run, one, queries, loop, Separately, querie...",1181,phpexcel


In [ ]:
tf.keras.